# Zadanie 3
### Interactions
Załóżmy, że jeśli Ania i Beata skomentowały ten sam film movie_1, dochodzi między nimi do bezpośredniej interakcji. Możemy sobie wyobrazić, że w pewien sposób rozmawiali ze sobą.
Jeśli Beata i Czarek wypowiedzieli się pod filmem movie_2, między nimi doszło również do interakcji.
W końcu Beata mogła wspomnieć w komentarzu pod filmem movie_2 Czarkowi o tym, czego dowiedziała się z komentarza Ani pod filmem movie_1. W ten sposób przyjmujemy, że pomiędzy Anią i Czarkiem pośrednio również doszło do interakcji.
Jeśli podzielimy w ten sposób wszystkich użytkowników na rozłączne grupy otrzymamy poszukiwane grupy interakcji. Co ważne – Ania będzie też w interakcji z każdą osobą, która pośrednio lub bezpośrednio jest w interakcji z Czarkiem.

#### Twoje zadanie polega na odpowiedzeniu ile mamy takich rozłącznych grup, jeśli:

o dataset
- nie ma dla nas znaczenia, czy user skomentował filmy należące do tej samej kategorii czy dotyczące tej samej gry. Istotne jest jedynie to, że doszło do interakcji w pewnych filmach.

o Group_game

- dzielimy użytkowników na grupy wyłącznie w oparciu o ich aktywność pod filmami wybranymi dla gier w tej kategorii. Tym samym Ania komentując filmy w dwóch różnych kategoriach będzie występowała w dwóch grupach. Liczba, której szukamy to faktycznie suma liczb rozłącznych grup interakcji dla wszystkich kategorii gier.

o Game
- posługując się tą samą logiką, co w przypadku group_game, szukamy sumy liczb rozłącznych grup interakcji dla wszystkich gier.

In [1]:
# Imports
import pandas as pd
#import numpy as np
from pandasql import sqldf

# SQL query function
def pysqldf(q):
    return sqldf(q, globals())
#    pysqldf = lambda q: sqldf(q, globals())

In [2]:
# folders, dataframes
data_path = 'data/' # data folder
data_frames  = ["comments","game_groups","game_movies","games"] #data csc files
output_folder = 'output/'# output folder

In [3]:
# read "zadania"
group_scores_Tomasz_Cwik = pd.read_csv("group_scores_name_surname.csv",encoding = 'utf-8',sep = ',')
game_scores_Tomasz_Cwik = pd.read_csv("game_scores_name_surname.csv",encoding = 'utf-8',sep = ',')
interactions_Tomasz_Cwik = pd.read_csv("interactions_name_surname.csv",encoding = 'utf-8',sep = ',')

#group_scores_Tomasz_Cwik 

In [4]:
# read tables
comments = pd.read_csv(data_path + "comments.csv", encoding = 'utf-8',sep = ',')
game_groups = pd.read_csv(data_path + "game_groups.csv", encoding = 'utf-8',sep = ',')
game_movies = pd.read_csv(data_path + "game_movies.csv",encoding = 'utf-8',sep = ',')
games = pd.read_csv(data_path + "games.csv",encoding = 'utf-8',sep = ',')

### Extra tables or variables

In [5]:
# create extra tables or variables

#list ot tabals or lists
# ----- distinct users ------df-------

# ---------- distinct users from comments

q = """
select distinct user from comments
"""
duser = pysqldf(q)

# ------- duser_number ---int------
duser_number =  duser['user'].count()
print("Total distinct useres in comments table: ",duser_number)
#duser # distinct users


# ---------- distinct movies Nodes from comments



Total distinct useres in comments table:  75429


# 3.2 Group_game
- dzielimy użytkowników na grupy wyłącznie w oparciu o ich aktywność pod filmami wybranymi dla gier w tej kategorii. Tym samym Ania komentując filmy w dwóch różnych kategoriach będzie występowała w dwóch grupach. Liczba, której szukamy to faktycznie suma liczb rozłącznych grup interakcji dla wszystkich kategorii gier.


In [6]:
# selct movies and game_groups
# nodes define table

q = """

   
   select c.id_movie, g.id_game, gm.title, gg.id_game_group, gg.game_group_name
   from
        (select distinct id_movie from comments ) as c 
        left join 
        game_movies as gm on c.id_movie = gm.id_movie
        left join 
        games as g on gm.id_game = g.id_game
        left join 
        game_groups as gg
        on g.id_game_group = gg.id_game_group
        
        
      

"""


movies_with_groups = pysqldf(q)
movies_with_groups
#movies_with_group.to_csv(path_or_buf  = "movies_with_group.csv", index = False, encoding = 'utf-8' )

,id_movie,id_game,title,id_game_group,game_group_name
0,1,1,Street Fighter - Justin vs Daigo,1,Fighting games
1,2,1,10 Batsh*t Crazy Things That Happened at Gamin...,1,Fighting games
2,3,1,The Most ICONIC Moments in EVO History (FGC),1,Fighting games
3,4,1,Esports,1,Fighting games
4,5,1,The Top 10 Bad Manner Moments in Esports,1,Fighting games
...,...,...,...,...,...
139,244,49,WGL EU - WoWs Showmatch | The Grand Finals 201...,8,Others
140,249,50,WoW Arena World Championship Grand Final,8,Others
141,251,51,Clash Royale League: OFFICIAL 2018 Season Kick...,8,Others
142,252,51,Orange Juice VS Nickatnyte | Clash Royale King...,8,Others


Logiga poszukiwania:

- Defniuję groupy stanowięce kategorię gry

następnie w ramach każdej kategorii gry przeprowadzam pętle podbnie jak w Zadaniu 3.1

- Definiuję węzeł "node" jako film (id_movie) z tabeli comments
- Szukamy interakcji zaczynając od dowolnego Noda (może to myć np. min(id_movie))
- Szukamy innych nodów (id_movie), które weszły w interakcję
- znalezione nody dodajemy do grupy znalezionych  - "zainfekowanych"
- zawężamy obszar poszukiwań o już przypisane nody
- powtarzamy interację do momentu przeszukania i przypisania wszystkich nodów

In [7]:
# defining groups

# define list groups of games
# we will search among movies limited to games from sama category
q = "Select distinct id_game from movies_with_groups"
game_groups_list = pysqldf(q)['id_game'].tolist()

print ("game_groups_list ", game_groups_list)


game_groups_list  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 26, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51]


In [8]:
# Main cell

# interactions dataset to stora data of interactions
interactions_dataset = pd.DataFrame(columns = ['id_interaction','id_node','type',"game_group_name","id_game_group"]) # here we stora info of interactions
interaction_groups_count = 0 # cunt the numner of groups search and found

#Loop for game_groups

for gg in game_groups_list:
    #print ("-",gg)
    
    # collect some information 
    q ="select * from games where id_game = " + str(gg)
    game_groups_names = pysqldf(q)
    game_groups_name_string = game_groups_names['game_name'].iloc[0] # w zad. 3.3 to nazwa gry
    game_groups_id_do_df = game_groups_names['id_game'].iloc[0] # w zad. 3.3 id gry
    #print ("game_groups_name_string", game_groups_name_string)
    
    #lets define nods (mvoies) in this group to search
    q = "Select distinct id_movie from movies_with_groups where id_game =  " + str(gg) # inaczej niż 3.2
    
    #print (q)
    game_group_nodes = pysqldf(q)['id_movie'].tolist()
    #print ("game_group_nodes # ", gg ," ", game_group_nodes)

    
    all_nodes_list = game_group_nodes
    
    #all_nodes_list_sql # string for sql search
    # lests make string for next iteration for SQL select
    
    all_nodes_list_sql = ""
    for x in all_nodes_list:
        all_nodes_list_sql += str(x)+","
    all_nodes_list_sql = "(" + all_nodes_list_sql[:-1] + ")"
    print ("all_nodes_list_sql",all_nodes_list_sql)

    
    # 3.1 start


    
    infected_nodes = [] # define a list of nodes we found, lets start with empty
    end_task_search = False # loop end variable
    end_task_search_count = 0 # Groups counter
    remain_nodes =  all_nodes_list # Definition of dataset to search - lets make put all data at start
    
    # prints
    print ("x"*60)
    print ("Game Category to search: ",game_groups_name_string, "id: ", game_groups_id_do_df  )
    print ("Games ids to search: All groups in dataset -group_game-: ", len(all_nodes_list))
    print (all_nodes_list)
    print ("x"*60)
    #print ("przed petla remain_nodes:", remain_nodes)
    #print ("przed petla all_nodes_list:", all_nodes_list)

    while end_task_search == False: # loop searchin the groups of interactions

        end_task_search_count +=1 

        # define nodes to search - name "search_node"
        
      

        remain_nodes = [x for x in remain_nodes if x not in infected_nodes]
        infected_nodes = [] # zerowanie infected notes
        
        #prints
        print ("-"*50)
        print ("")
        print (">> Searching group #:  ", interaction_groups_count)
        print ("Group ID: ", game_groups_id_do_df )
        print ("Group category: ", game_groups_name_string )
       


        if remain_nodes: # testing if we should end the process
            print ("")
            interaction_groups_count +=1
        else:
            print ("end of search")
            end_task_search = False
            break

      
        print ("Group of interactions #: ", interaction_groups_count)
        print ("Nodes infected",infected_nodes)
        print ("Ramaing nodes to check ",remain_nodes)
        print ("")

        search_node = "(" + str(remain_nodes[0]) + ")"
        infected_nodes = [remain_nodes[0]] # definiuje pierwszego szukanego jako zarażonego 

        #test if we switch the order
        #if interaction_groups_count == 1:
        #    search_node = "(" + str(remain_nodes[122]) + ")"
        #    infected_nodes = [remain_nodes[122]]

        #print ("Search nodes: ",search_node)


        infected_nodes_sql = search_node
        #print (search_node)
        # Loop for seraching interactions (start from no 1)
        # zeros of 
        i=0
        end_graph_search = False


        # -- Loop for seraching interactions (start from no 1)----------------------------------------------------------------------------------------------------------------
        while end_graph_search == False:
            i= i + 1
            print ("---------------------")
            print (">> Searching group #:  ", interaction_groups_count, " Iteration #: ", i)
            print ("Seach nodes: ", search_node )
            #print ("---------------------")
            

            #print ("przed sql remain_nodes:", remain_nodes)
            #print ("przed sql all_nodes_list:", all_nodes_list)
    
            # step 1

            # step 1.1 Search all users that could have been infected with Node
            # That case is id_movie
            q_step_1 = """
                select distinct user
                from comments
                where id_movie in """ + str(search_node) + """

                """
            infected_users = pysqldf(q_step_1)
            #print (q)
            #Step 2
            # Search of nodes that are infected with useres from the previous node 
            
            
            #all_nodes_list_sql # string for sql search
            # lests make string for next iteration for SQL select

            #all_nodes_list_sql = ""
            
            #all_nodes_minus_sarch_node = [x for x in all_nodes_list if x not in infected_nodes]
            #remain_nodes = [x for x in remain_nodes if x not in infected_nodes]
            
            #for x in all_nodes_list:
            #    all_nodes_list_sql += str(x)+","
            #all_nodes_list_sql = "(" + all_nodes_list_sql[:-1] + ")"
            #print ("all_nodes_list_sql",all_nodes_list_sql)
            

            q= """
                with next_infected_nodes_search as (

                    select distinct id_movie 
                    from comments 
                        where
                        user in (

                                select distinct user
                                from comments
                                where 
                                    id_movie in """ + str(search_node) + """
                                



                                )   
                                                )
                select id_movie from next_infected_nodes_search
                    where id_movie NOT IN """ + infected_nodes_sql + """ /* wothout already found */
                    AND id_movie IN """ + all_nodes_list_sql + """ /* wothout already found */
                   
                """                     


            #print(q)
            next_infected_nodes = pysqldf(q)

            # NEXT INFECTED
            # ----------------------------------------------------------------------
            #next_infected_nodes found list
            next_infected_nodes_list = next_infected_nodes['id_movie'].tolist()

            # lests make string for next iteration for SQL select
            search_node =""
            for x in next_infected_nodes_list:
                search_node += str(x)+","
            search_node = "(" + search_node[:-1] + ")"
            #print ("Tomek search node", search_node) # intermidiate test

            # lets count how many was found
            next_infected_nodes_list_count = len(next_infected_nodes_list)

            # Add infected nodes 
            infected_nodes = infected_nodes +  next_infected_nodes_list 
            #print ("infected_nodes + test next_infected_nodes_list ",infected_nodes) # Test print

            # lets make string for SQL of infected_nodes
            infected_nodes_sql =""
            for x in infected_nodes:
                infected_nodes_sql += str(x)+","
            infected_nodes_sql = "(" + infected_nodes_sql[:-1] + ")"


            # If to check if we should stop the searchin becasue no more nodes are found
            if next_infected_nodes_list_count == 0:
                # nothing was found we stop to loop
                print ()
                print ("Did not find any relationship /any infected nodes/...")
                print ("End of iterations")
                print ("----------------------- End Result -----------------------------------")
                print ("Numner of group: ", interaction_groups_count)
                print ("Infected nodes founded: " , len(infected_nodes) )
                print ("List of founded: ")
                print (infected_nodes)

                print ("Id of group of interactions: " , end_task_search_count)
                print ("----------------------------------------------------------------------")

                end_graph_search = True

            else:
                # There are still nodes to search
                print()
                print ("Infected nodes found: ", next_infected_nodes_list_count)
                print ()
                print ("List of found and infected: ")
                print (next_infected_nodes_list)
                print ()
                print ("So far total identified: " , len(infected_nodes) )
                print (infected_nodes)
                print ("continue searching")

            # end of the loop


        # Building a datafreame with info
        f=0
        #print ("test len range", len(infected_nodes))
        #print ("test len range od 0 ", infected_nodes[0])
        for f in range(len(infected_nodes)):
            # Append Row to DataFrame
            #print ("f: ",f, " i: ", i)

            list_row = [interaction_groups_count, infected_nodes[f], "game", game_groups_name_string , game_groups_id_do_df]
            interactions_dataset.loc[len(interactions_dataset)] = list_row


        interactions_dataset



all_nodes_list_sql (1,2,3,4,5)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Game Category to search:  Street Fighter id:  1
Games ids to search: All groups in dataset -group_game-:  5
[1, 2, 3, 4, 5]
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
--------------------------------------------------

>> Searching group #:   0
Group ID:  1
Group category:  Street Fighter

Group of interactions #:  1
Nodes infected []
Ramaing nodes to check  [1, 2, 3, 4, 5]

---------------------
>> Searching group #:   1  Iteration #:  1
Seach nodes:  (1)

Infected nodes found:  3

List of found and infected: 
[3, 4, 5]

So far total identified:  4
[1, 3, 4, 5]
continue searching
---------------------
>> Searching group #:   1  Iteration #:  2
Seach nodes:  (3,4,5)

Infected nodes found:  1

List of found and infected: 
[2]

So far total identified:  5
[1, 3, 4, 5, 2]
continue searching
---------------------
>> Searching group #:   1  Iteration #:  3
Seach nodes:  (2)

Did not

In [9]:
interactions_dataset

,id_interaction,id_node,type,game_group_name,id_game_group
0,1,1,game,Street Fighter,1
1,1,3,game,Street Fighter,1
2,1,4,game,Street Fighter,1
3,1,5,game,Street Fighter,1
4,1,2,game,Street Fighter,1
...,...,...,...,...,...
139,69,244,game,World of Tanks,49
140,70,249,game,World of Warcraft,50
141,71,251,game,Clash Royale,51
142,71,252,game,Clash Royale,51


In [10]:
#infected_nodes
#nodes_dataset = pd.DataFrame(infected_nodes , columns= ['id_movie'])
interactions_dataset

interactions_dataset.to_csv(path_or_buf  = "Zadanie_3_3_Tomasz_Cwik_interactions_dataset_game_group-lista_grup.csv", index = False, encoding = 'utf-8' )
#del nodes_dataset

In [11]:
# Making and saving list of groups with users

q = """

Select i.id_node, c.id_movie, c.user
    from interactions_dataset as i
        left join (
                    select distinct id_movie, user from comments
                     ) as c
        
        on i.id_node = c.id_movie
        

"""
interaction_users = pysqldf(q)
interaction_users
interaction_users.to_csv(path_or_buf  = "Zadanie_3_3_interaction_users_groups_Tomasz_Cwik.csv", index = False, encoding = 'utf-8' )